In [460]:
from pathlib import Path
import importlib
import warnings
import time
import os
from datetime import datetime
import time
warnings.filterwarnings("ignore")  # Oculta todos los warnings

# Mis modulos
import process_data as dp
import transform_data as td
import html_generator as hg

In [461]:
# import pyfiglet
# print(pyfiglet.figlet_format("Gestor de Vacaciones"))

banner = r"""
  ____           _                   _      
 / ___| ___  ___| |_ ___  _ __    __| | ___ 
| |  _ / _ \/ __| __/ _ \| '__|  / _` |/ _ \
| |_| |  __/\__ \ || (_) | |    | (_| |  __/
 \____|\___||___/\__\___/|_|     \__,_|\___|
                                            
__     __                   _                       
\ \   / /_ _  ___ __ _  ___(_) ___  _ __   ___  ___ 
 \ \ / / _` |/ __/ _` |/ __| |/ _ \| '_ \ / _ \/ __|
  \ V / (_| | (_| (_| | (__| | (_) | | | |  __/\__ \
   \_/ \__,_|\___\__,_|\___|_|\___/|_| |_|\___||___/

    💥 VISUALIZADOR Y EDITOR DE VACACIONES 💥
"""

print(banner, end='\n\n')
time.sleep(1)


  ____           _                   _      
 / ___| ___  ___| |_ ___  _ __    __| | ___ 
| |  _ / _ \/ __| __/ _ \| '__|  / _` |/ _ \
| |_| |  __/\__ \ || (_) | |    | (_| |  __/
 \____|\___||___/\__\___/|_|     \__,_|\___|

__     __                   _                       
\ \   / /_ _  ___ __ _  ___(_) ___  _ __   ___  ___ 
 \ \ / / _` |/ __/ _` |/ __| |/ _ \| '_ \ / _ \/ __|
  \ V / (_| | (_| (_| | (__| | (_) | | | |  __/\__ \
   \_/ \__,_|\___\__,_|\___|_|\___/|_| |_|\___||___/

    💥 VISUALIZADOR Y EDITOR DE VACACIONES 💥




In [462]:
# Constantes
this_year = datetime.today().date().year

# Diccionarios
months = {
    1: 'Enero', 2: 'Febrero', 3: 'Marzo', 4: 'Abril', 5: 'Mayo', 6: 'Junio',
    7: 'Julio', 8: 'Agosto', 9: 'Septiembre', 10: 'Octubre', 
    11: 'Noviembre', 12: 'Diciembre'
}
vacaciones = {
    "name": "Gestor de Vacaciones",
    "file_name_xlsx": "DATOS_VACACIONES_2020-2025.xlsx",
    "file_name_csv": "vacaciones_data.csv",
    "sheet_name": "DATOS",
    "relevant_columns": [
        'APELLIDOS',
        'NOMBRES',
        'DNI',
        'CARGO',
        'Fecha Ingreso',
        'Vacaciones 2020-2021',
        'Vacaciones 2021-2022',
        'Vacaciones 2022-2023',
        'Vacaciones 2023-2024',
        'Vacaciones 2024-2025',
    ],
    "new_columns": [
        "DIAS_ACUMULADOS",
        "VACACIONES_GOZADAS",
        "VACACION_GOZADA_ACTUAL", # 3 estados (No vacaciono, esta vacacionando, por vacacionar)
        "VACACIONES_ACUMULADAS", # Dias de vacaciones acumuladas que tiene pendiente para gozar
    ]
}
VACACION_GOZADA_ACTUAL_ESTADOS = {
    0: f"Vacaciones no aplicables",
    1: f"Vacaciones del {this_year} gozadas",
    2: "Gozando vacaciones actualmente",
    3: f"Vacaciones del {this_year} por gozar",
}

# Listas

# Constantes
project_address = r"C:\Informacion\Otros\RRHH\Automatizacion\vacation_manager"

### Procesamiento de datos

In [463]:
importlib.reload(dp)

df = dp.main(project_address, vacaciones)

### Transformacion de datos

In [464]:
importlib.reload(td)

df = td.main(df)

### Generar reporte personal

In [474]:
import polars as pl
with pl.Config(tbl_cols=-1):
    print(df.filter(pl.col("DNI") == 0000000))

importlib.reload(hg)

html_resultado = hg.main(project_address, df, months, VACACION_GOZADA_ACTUAL_ESTADOS, 00000000)

shape: (1, 21)
┌─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┐
│ APE ┆ NOM ┆ DNI ┆ CAR ┆ Fec ┆ Vac ┆ Vac ┆ Vac ┆ Vac ┆ Vac ┆ NOM ┆ DIA ┆ VAC ┆ VAC ┆ VAC ┆ DIA ┆ ALE ┆ ANI ┆ PRO ┆ DIA ┆ ALE │
│ LLI ┆ BRE ┆ --- ┆ GO  ┆ ha  ┆ aci ┆ aci ┆ aci ┆ aci ┆ aci ┆ BRE ┆ S_A ┆ ACI ┆ ACI ┆ ACI ┆ S_P ┆ RTA ┆ VER ┆ XIM ┆ S_P ┆ RTA │
│ DOS ┆ S   ┆ i64 ┆ --- ┆ Ing ┆ one ┆ one ┆ one ┆ one ┆ one ┆ _CO ┆ CUM ┆ ONE ┆ ON_ ┆ ONE ┆ ARA ┆ _VA ┆ SAR ┆ O_A ┆ ARA ┆ _AN │
│ --- ┆ --- ┆     ┆ str ┆ res ┆ s   ┆ s   ┆ s   ┆ s   ┆ s   ┆ MPL ┆ ULA ┆ S_G ┆ GOZ ┆ S_A ┆ _VA ┆ CAC ┆ IO_ ┆ NIV ┆ _AN ┆ IVE │
│ str ┆ str ┆     ┆     ┆ o   ┆ 202 ┆ 202 ┆ 202 ┆ 202 ┆ 202 ┆ ETO ┆ DOS ┆ OZA ┆ ADA ┆ CUM ┆ CAC ┆ ION ┆ TEM ┆ ERS ┆ IVE ┆ RSA │
│     ┆     ┆     ┆     ┆ --- ┆ 0-2 ┆ 1-2 ┆ 2-2 ┆ 3-2 ┆ 4-2 ┆ --- ┆ --- ┆ DAS ┆ _AC ┆ ULA ┆ ION ┆ ES  ┆ P   ┆ ARI ┆ RSA ┆ RIO │
│     ┆     ┆     ┆     ┆ dat ┆ 021 ┆ 022 ┆ 023 ┆ 024 ┆ 025 ┆ str ┆ i64 ┆ --- ┆ TUA ┆ DAS

### Reporte de Alerta 1 (a una semana a entrar a vacaciones)

### Reporte de Alerta 2 (a una semana de cumplir aniversario)